In [ ]:
from PSUTester import *
p=PSUTester('fastTest.yaml')
#Test flow
p.test_flow()



In [ ]:
settings=p.config['DUT']
p.input_configuration(settings['input'])
sequence_checks=[]
for i,settings in settings['output'].items():
    #p.run_unit_test(settings)
    if settings['sequence_check']==True:
        sequence_checks.append(settings['channel'])
print(sequence_checks)

p.oscilloscope.write("ACQuire:STATE RUN")
p.oscilloscope.write("ACQuire:STOPAfter RUNSTOP")
p.power_supply.off()
p.load.off()

#Switch on channels
channel_in=p.config['DUT']['input']['channel']
in_scale=p.oscilloscope.nearest_v_scale(p.config['DUT']['input']['voltage'])
p.oscilloscope.channel_on(channel_in)
p.oscilloscope.set_y_scale(channel_in,in_scale)
for i in sequence_checks:
    p.oscilloscope.channel_on(i)
    p.oscilloscope.set_y_scale(i,in_scale)

In [ ]:

p.oscilloscope.set_t_scale(0.01)
p.oscilloscope.write("HORIZONTAL:MAIN:DELAY:TIME 2e-2") 
#Config oscilloscope to single mode rise trigger
p.oscilloscope.write(f"TRIGGER:A:EDGE:SOURCE CH{channel_in}")
p.oscilloscope.write(f"TRIGGER:A:LEVel {p.config['DUT']['input']['voltage']*0.1}")
p.oscilloscope.write("TRIGGER:A:EDGE:SLOPE RISE") 
p.oscilloscope.write("ACQuire:STOPAfter SEQUENCE")
p.oscilloscope.write("ACQuire:STATE RUN")
time.sleep(5)
#Power on sequence
p.power_supply.on()
time.sleep(2)
p.oscilloscope.get_waveform_all()
#p.power_supply.off()



In [ ]:
p.oscilloscope.write(f"TRIGGER:A:LEVel {p.config['DUT']['input']['voltage']*0.5}")
p.oscilloscope.write("TRIGGER:A:EDGE:SLOPE FALL") 
p.oscilloscope.write("ACQuire:STOPAfter SEQUENCE")
p.oscilloscope.write("ACQuire:STATE RUN")


In [ ]:
time.sleep(2)
p.power_supply.off()
time.sleep(2)
p.oscilloscope.get_waveform_all()
time.sleep(2)
p.oscilloscope.write("ACQuire:STATE RUN")
p.oscilloscope.write("ACQuire:STOPAfter RUNSTOP")

In [ ]:
p.oscilloscope.get_frequency(2)

In [ ]:
def nearest_scale(value):
    scales = [1, 2, 4]
    powers = [-9, -8, -7, -6, -5, -4, -3,-2,-1,0]  # 10的n次方

    nearest_diff = float('inf')
    nearest_scale = None

    for scale in scales:
        for power in powers:
            scale_value = scale * 10 ** power
            diff = abs(value - scale_value)
            if diff < nearest_diff:
                nearest_diff = diff
                nearest_scale = scale_value

    return nearest_scale

# 测试示例
value = 1/1
nearest = nearest_scale(value)
print(f"Value: {value}, Nearest Scale: {nearest}")


In [ ]:
new_s=nearest_scale(1/p.oscilloscope.get_measurement(4))
print(new_s)
p.oscilloscope.set_t_scale(new_s)